In [ ]:
# delete all with fewer than 4 keyframes or has no status.txt file

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re

# Set the root directory
root_dir = "recorded_data"

# Collect all successful tasks
tasks = []
for x_dir in os.listdir(root_dir):
    x_path = os.path.join(root_dir, x_dir)
    status_file = os.path.join(x_path, "status.txt")
    image_dir = os.path.join(x_path, "task/Final_Image")

    if os.path.isdir(x_path) and os.path.isfile(status_file) and os.path.isdir(image_dir):
        with open(status_file, "r") as file:
            if file.read().strip() == "success":
                # Find all images matching the pattern final_image_<number>.jpg
                for image_file in os.listdir(image_dir):
                    if re.match(r"final_image_\d+\.jpg", image_file):
                        tasks.append((x_dir, os.path.join(image_dir, image_file), status_file))

def show_images_inline(tasks):
    """Displays each image inline using matplotlib in Jupyter Notebook."""
    for task in tasks:
        img = mpimg.imread(task[1])
        plt.imshow(img)
        plt.title(f"Task {task[0]}")
        plt.axis('off')
        plt.show()

print(len(tasks))
show_images_inline(tasks)



In [4]:
import os
import shutil

def merge_folders(folder1, folder2, target_folder):
    # Ensure the target folder exists
    os.makedirs(target_folder, exist_ok=True)

    # Helper function to get the next available integer name
    def get_next_name(existing_names):
        i = 1
        while str(i) in existing_names:
            i += 1
        return str(i)

    # Collect all subfolder names already in the target folder
    existing_names = set(os.listdir(target_folder))

    # Process each folder
    for folder in [folder1, folder2]:
        for subfolder in os.listdir(folder):
            source_path = os.path.join(folder, subfolder)
            if os.path.isdir(source_path):  # Ensure it's a subfolder
                if subfolder in existing_names:
                    # Resolve conflict by renaming
                    new_name = get_next_name(existing_names)
                    existing_names.add(new_name)
                    target_path = os.path.join(target_folder, new_name)
                else:
                    existing_names.add(subfolder)
                    target_path = os.path.join(target_folder, subfolder)

                # Move the subfolder
                shutil.move(source_path, target_path)
                print(f"Moved '{source_path}' to '{target_path}'")

# Example usage
folder1 = "./recorded_data"
folder2 = "./backup"
target_folder = "./merged_data"

merge_folders(folder1, folder2, target_folder)